In [1]:
# R to Python conversion of Cox simulation setup
import numpy as np
from scipy.stats import multivariate_normal, expon
from sklearn.preprocessing import StandardScaler

# Set parameters (matching R code exactly)
n = 400
p = 200
nonzero_coefs = 30
Amp = 5
rho = 0.6

# Generate Toeplitz correlation matrix (equivalent to toeplitz(rho^(0:(p-1))))
Theta_8 = np.array([[rho**abs(i-j) for j in range(p)] for i in range(p)])

# Generate X from multivariate normal (equivalent to mvrnorm)
X = multivariate_normal.rvs(mean=np.zeros(p), cov=Theta_8, size=n, random_state=42)

# Scale X with 1/sqrt(n) factor (equivalent to scale(X)/(sqrt(n)))
X = StandardScaler().fit_transform(X) / np.sqrt(n)

# Generate true coefficients (equivalent to beta setup)
beta = np.zeros(p)
beta[:nonzero_coefs] = Amp  # All positive coefficients in Cox model

# Signal indices (equivalent to Signal_index <- 1:nonzero_coefs)
Signal_index = np.arange(nonzero_coefs)  # 0-indexed in Python vs 1-indexed in R

# True labels (equivalent to true_labels <- beta != 0)
true_labels = beta != 0

# Generate survival times using exponential distribution
# This is a simplified version of simsurv - in practice you'd use more sophisticated survival simulation
# For Cox model, we use exponential baseline hazard with rate = exp(X @ beta)
hazard_rates = np.exp(X @ beta)
surv_times = np.random.exponential(1.0 / hazard_rates)

# All events are observed (no censoring for simplicity)
status = np.ones(n, dtype=int)
time = surv_times

# Create survival data in the format expected by scikit-survival
y = np.column_stack([status, time])

print(f"Data shape: X={X.shape}, y={y.shape}")
print(f"True signal indices: {Signal_index}")
print(f"Number of true signals: {np.sum(true_labels)}")
print(f"Survival time range: [{np.min(time):.3f}, {np.max(time):.3f}]")
print(f"All events observed: {np.sum(status)}/{n} ({np.mean(status):.3f} proportion)")
print(f"Mean survival time: {np.mean(time):.3f}")
print(f"Median survival time: {np.median(time):.3f}")


Data shape: X=(400, 200), y=(400, 2)
True signal indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
Number of true signals: 30
Survival time range: [0.000, 3936.339]
All events observed: 400/400 (1.000 proportion)
Mean survival time: 30.600
Median survival time: 0.597


In [2]:
from nullstrap.models.cox import NullstrapCox

# After the simulation code above...

# Initialize Nullstrap Cox
model = NullstrapCox(
    fdr=0.1,           # Target FDR
    alpha_=None,       # Auto-select alpha (regularization parameter)
    B_reps=5,          # Bootstrap repetitions
    random_state=42    # For reproducibility
)

# Fit the model
model.fit(X, y)

# Get results
threshold = model.threshold_
selected_features = model.selected_
statistics = model.statistic_
correction_factor = model.correction_factor_
alpha_used = model.alpha_used_

print(f"Threshold: {threshold}")
print(f"Selected features: {selected_features}")
print(f"Number selected: {len(selected_features)}")
print(f"Correction factor: {correction_factor}")
print(f"Alpha used: {alpha_used}")


/Users/promptbio/Library/CloudStorage/OneDrive-Personal/code/pyNullstrap/nullstrap/models/cox.py:280: UserWarning: Cox model received 2D array. Auto-converting from [event, time] format to structured array.
  warnings.warn(
/Users/promptbio/Library/CloudStorage/OneDrive-Personal/code/pyNullstrap/nullstrap/models/cox.py:324: UserWarning: All observations are events - no censoring detected
  warnings.warn("All observations are events - no censoring detected", UserWarning)


Cox model inference: [event, time], time range: [0.000, 3936.339], events: 400.0/400 (1.000)
Threshold: 0.026929429674342864
Selected features: [  0   1   6   8  11  14  16  17  18  20  21  23  25  27  28  34  42  45
  49  57  60  61  62  64  72  76  89  91 106 111 112 116 119 127 128 129
 131 132 133 139 144 145 154 158 159 161 163 170 172 173 176 180 183 185
 187 190 193 197 199]
Number selected: 59
Correction factor: 0.050000009819876415
Alpha used: 0.42349807599143463


In [3]:
# Note: For Cox model comparison, we'll use Lasso on survival times as a baseline
# since scikit-survival doesn't have built-in L1 regularization for Cox models
from sklearn.linear_model import LassoCV
import numpy as np

# Fit Lasso on survival times as baseline comparison
lasso = LassoCV(cv=5, random_state=42).fit(X, time)

# Get the coefficients
lasso_coef = lasso.coef_

# Select features with nonzero coefficients
lasso_selected = np.where(lasso_coef != 0)[0]

print(f"Lasso selected features: {lasso_selected}")
print(f"Number selected by Lasso: {len(lasso_selected)}")
print(f"Lasso alpha (lambda) used: {lasso.alpha_}")

# Alternative: Use Cox model from scikit-survival for comparison
try:
    from sksurv.linear_model import CoxPHSurvivalAnalysis
    from sksurv.util import Surv
    
    # Convert to scikit-survival format
    y_sksurv = np.array([(bool(status[i]), time[i]) for i in range(n)], 
                       dtype=[('event', bool), ('time', float)])
    
    # Fit Cox model without regularization
    cox_baseline = CoxPHSurvivalAnalysis(alpha=0.0)
    cox_baseline.fit(X, y_sksurv)
    
    # Get coefficients and select features
    cox_coef = cox_baseline.coef_
    cox_selected = np.where(np.abs(cox_coef) > 1e-6)[0]
    
    print(f"\nCox baseline selected features: {cox_selected}")
    print(f"Number selected by Cox baseline: {len(cox_selected)}")
    
except ImportError:
    print("\nNote: scikit-survival not available for Cox baseline comparison")
    cox_selected = np.array([])
    


Lasso selected features: [6]
Number selected by Lasso: 1
Lasso alpha (lambda) used: 2.4122754212743107

Cox baseline selected features: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199]
Number selected by Cox baseline: 200


In [4]:
from nullstrap.utils.metrics import compute_power, compute_fdr

# Compute true support
true_support = np.where(beta != 0)[0]

# Nullstrap Cox results
nullstrap_power = compute_power(selected_features, true_support)
nullstrap_fdr = compute_fdr(selected_features, true_support, p) 
print(f"Nullstrap Cox Power: {nullstrap_power:.3f}")
print(f"Nullstrap Cox FDR: {nullstrap_fdr:.3f}")

# Lasso results
lasso_power = compute_power(lasso_selected, true_support)
lasso_fdr = compute_fdr(lasso_selected, true_support, p)
print(f"Lasso Power: {lasso_power:.3f}")
print(f"Lasso FDR: {lasso_fdr:.3f}")

# Cox baseline results (if available)
if len(cox_selected) > 0:
    cox_power = compute_power(cox_selected, true_support)
    cox_fdr = compute_fdr(cox_selected, true_support, p)
    print(f"Cox Baseline Power: {cox_power:.3f}")
    print(f"Cox Baseline FDR: {cox_fdr:.3f}")

# Additional Cox-specific metrics
print(f"\nSurvival data summary:")
print(f"  - Sample size: {n}")
print(f"  - All events observed: {np.sum(status)}/{n} ({np.mean(status):.3f} proportion)")
print(f"  - Survival time range: [{np.min(time):.3f}, {np.max(time):.3f}]")
print(f"  - Mean survival time: {np.mean(time):.3f}")
print(f"  - Median survival time: {np.median(time):.3f}")
print(f"  - True positive rate: {np.sum(selected_features < nonzero_coefs)}/{nonzero_coefs} true signals selected")
print(f"  - False positive rate: {np.sum(selected_features >= nonzero_coefs)}/{len(selected_features)} selected features are false positives")


Nullstrap Cox Power: 0.500
Nullstrap Cox FDR: 0.746
Lasso Power: 0.033
Lasso FDR: 0.000
Cox Baseline Power: 1.000
Cox Baseline FDR: 0.850

Survival data summary:
  - Sample size: 400
  - All events observed: 400/400 (1.000 proportion)
  - Survival time range: [0.000, 3936.339]
  - Mean survival time: 30.600
  - Median survival time: 0.597
  - True positive rate: 15/30 true signals selected
  - False positive rate: 44/59 selected features are false positives
